In [12]:
import pandas as pd
import numpy as np
import re

In [14]:
valid_categories = [
    'Food Not Available',
    'Restaurant Closed',
    'Customer wants to cancel',
    'Wrong location',
    'Wrong Order',
    'Out of zone',
    'User unreachable',
    'Restaurant didn\'t pick call',
    'Test Order',
    'Rider arriving delay',
    'Rider Shortage',
    'Need overtime for food processing',
    'Resturaunt Internal Issues',
    'Wrong Branch',
    'Admin Issues',
    'Rider didn\'t pick call',
    'Rider Assigned Delay',
    'Weather Issues',
    'Zone Closed',
    'Customer\'s Payment Method',
    'Rider issue',
    'Rider Didn\'t come',
    'Dark Zone',
    'Other'
]

In [29]:
def classify(remarks, cancel_reason):
    """
    Comprehensive classification using ALL categories
    Priority: Most specific match wins
    """
    
    text = str(remarks).lower().strip()
    reason = str(cancel_reason).lower().strip()
    
    # ========== LOCATION ISSUES (CHECK FIRST) ==========
    # "Resto location bhul" = Restaurant location wrong
    if any(pattern in text for pattern in [
        'location bhul',
        'location vul',
        'wrong location',
        'wrong address',
        'incorrect location',
        'incorrect address',
        'address mistake',
        'location mistake',
        'address error',
        'location error',
        'wrong place',
        'vul address',
        'bhul address',
        'address issue',
        'location issue',
        'wrong branch location',
        'wrong resto location'
    ]):
        # Check if it's about wrong branch specifically
        if 'branch' in text:
            return 'Wrong Branch'
        return 'Wrong location'
    
    # ========== RESTAURANT COMMUNICATION ISSUES ==========
    # Restaurant/Resto number off, unreachable, didn't pick
    if any(pattern in text for pattern in [
        "restaurant didn't pick",
        "restaurant didnt pick",
        "resto didn't pick",
        "resto didnt pick",
        "restaurant not pick",
        "resto not pick",
        "restaurant unreachable",
        "resto unreachable",
        "restaurant dnp",
        "resto dnp",
        "restaurant number off",
        "resto number off",
        "restaurant's number is off",
        "resto's number is off",
        "restaurant phone off",
        "restaurant not answering",
        "resto not answering",
        "can't contact restaurant",
        "cannot contact restaurant",
        "restaurant no response",
        "resto no response"
    ]) and reason in "restaurand_issue":
        return "Restaurant didn't pick call"
    
    # ========== RIDER COMMUNICATION ISSUES ==========
    if any(pattern in text for pattern in [
        "rider didn't pick",
        "rider didnt pick",
        "rider not pick",
        "rider unreachable",
        "rider dnp",
        "rider number off",
        "rider phone off",
        "rider not answering",
        "can't contact rider",
        "rider no response"
    ]):
        return "Rider didn't pick call"
    
    # ========== FOOD NOT AVAILABLE ==========
    if any(pattern in text for pattern in [
        'unavailable',
        'not available',
        'item nei',
        'food nei',
        'stock out',
        'sold out',
        'out of stock',
        'finished',
        'items are not',
        'item are not',
        'item is not available',
        'food is not available',
        'not in stock',
        'shesh',
        'nai',
        'item' 
    ]):
        return 'Food Not Available'
    
    # ========== RESTAURANT CLOSED ==========
    if any(pattern in text for pattern in [
        'closed',
        'restaurant off',
        'resto off',
        'restaurant is off',
        'turned off',
        'turn off',
        'shop off',
        'not open',
        'outlet closed',
        'bondho',
        'restaurant close',
        'resto close',
        'shop close'
    ]):
        # Check if it's zone closed specifically
        if 'zone' in text:
            return 'Zone Closed'
        return 'Restaurant Closed'
    
    # ========== ZONE ISSUES ==========
    if any(pattern in text for pattern in [
        'zone closed',
        'zone off',
        'zone issue',
        'delivery zone',
        'dark zone',
        'zone problem'
    ]):
        if 'dark' in text:
            return 'Dark Zone'
        elif 'closed' in text or 'off' in text:
            return 'Zone Closed'
        else:
            return 'Out of zone'
    
    if any(pattern in text for pattern in [
        'out of zone',
        'outside zone',
        'beyond zone',
        'zone limit',
        'outside delivery',
        'delivery area',
        'out of coverage',
        'zone boundary'
    ]):
        return 'Out of zone'
    
    # ========== CUSTOMER WANTS TO CANCEL ==========
    if any(pattern in text for pattern in [
        'customer wants to cancel',
        'customer want to cancel',
        'customer wanted to cancel',
        'wants to cancel',
        'want to cancel',
        'customer cancel',
        'customer asked',
        'customer request',
        'changed mind',
        'change mind',
        'requested cancel',
        'asked to cancel',
        'user asked',
        'chai na',
        'dorkar nai',
        'baby order',
        'mistakenly ordered',
        'duplicate order',
        'double order'
    ]) and reason in "customer_issue":
        return 'Customer wants to cancel'
    
    # ========== USER UNREACHABLE ==========
    if any(pattern in text for pattern in [
        "customer didn't pick",
        "customer didnt pick",
        "customer did not pick",
        "customer not pick",
        "user didn't pick",
        "user didnt pick",
        "didn't pick up the call",
        "didnt pick up the call",
        "unreachable",
        "phone off",
        "switched off",
        "number off",
        "number is off",
        "not answering",
        "no response",
        "customer dnp",
        "user dnp"
    ]):
        # Make sure it's about customer/user
        if 'restaurant' not in text and 'resto' not in text and 'rider' not in text:
            return 'User unreachable'
    
    # ========== WRONG ORDER ==========
    if any(pattern in text for pattern in [
        'wrong order',
        'wrong item',
        'incorrect order',
        'incorrect item',
        'different item',
        'mistake order',
        'wrong food',
        'wrong product',
        'order mistake',
        'item mistake'
    ]):
        return 'Wrong Order'
    
    # ========== RIDER DELAY ISSUES ==========
    if any(pattern in text for pattern in [
        'rider delay',
        'rider late',
        'delivery delay',
        'arriving delay',
        'late delivery',
        'rider slow',
        'waiting long',
        'taking too long',
        'deri',
        'rider took time',
        'rider response delay',
        'customer asked to cancel the order due to the delay'
    ]):
        return 'Rider arriving delay'
    
    # ========== RIDER DELAY ISSUES ==========
    if any(pattern in text for pattern in [
        'customer asked to cancel the order due to the delay',
        'delay',
        'due to delay'
    ]) and reason in "rider_issue":
        return 'Rider arriving delay'        
    # ========== RIDER ASSIGNMENT ISSUES ==========
    if any(pattern in text for pattern in [
        'rider not assigned',
        'rider assignment',
        'no rider assigned',
        'rider allocation',
        'pending rider',
        'waiting for rider',
        'rider assign delay',
        'rider assigned delay',
        'assignment delay',
        're-assign',
        'reassign'
    ]):
        return 'Rider Assigned Delay'
    
    # ========== RIDER SHORTAGE ==========
    if any(pattern in text for pattern in [
        'rider shortage',
        'no rider',
        'rider not available',
        'rider unavailable',
        'no available rider',
        'rider busy',
        'all rider busy',
        'insufficient rider'
    ]):
        return 'Rider Shortage'
    
    # ========== RIDER DIDN'T COME ==========
    if any(pattern in text for pattern in [
        "rider didn't come",
        "rider didnt come",
        "rider did not come",
        "rider not come",
        "rider didn't arrive",
        "rider not arrived",
        "rider didn't show"
    ]):
        return "Rider Didn't come"
    
    # ========== RIDER GENERAL ISSUES ==========
    if 'rider issue' in text or 'rider problem' in text:
        return 'Rider issue'
    
    # ========== TEST ORDER ==========
    if any(pattern in text for pattern in [
        'test order',
        'testing order',
        'demo order',
        'trial order',
        'dummy order',
        'test purpose'
    ]):
        return 'Test Order'
    
    # ========== RESTAURANT INTERNAL ISSUES ==========
    if any(pattern in text for pattern in [
        'internal issue',
        'kitchen issue',
        'kitchen problem',
        'restaurant issue',
        'resto issue',
        'technical issue',
        'electricity issue',
        'chef issue',
        'chef sick',
        'no electricity',
        'power issue',
        'system issue',
        'internet'
    ]): 
        return 'Resturaunt Internal Issues'


    # ========== NEED OVERTIME ==========
    if any(pattern in text for pattern in [
        'more time',
        'overtime',
        'time lagbe',
        'preparation late',
        'will take time',
        'takes time',
        'need time',
        '1 hour',
        '1.5 hour',
        '2 hour',
        'huge delay',
        'long preparation'
    ]):
        return 'Need overtime for food processing'


    
    if any(pattern in text for pattern in [
        'customer asked to cancel the order due to the delay',
        'delay',
        'due to delay'
    ]) and reason in "restaurant_issue":
        return "Need overtime for food processing"
    
    # ========== WEATHER ISSUES ==========
    if any(pattern in text for pattern in [
        'weather',
        'rain',
        'storm',
        'flood',
        'heavy rain',
        'bad weather',
        'weather issue',
        'weather problem'
    ]):
        return 'Weather Issues'
    
    # ========== PAYMENT ISSUES ==========
    if any(pattern in text for pattern in [
        'payment',
        'card declined',
        'transaction fail',
        'payment fail',
        'payment issue',
        'payment method',
        'payment problem',
        'cod',
        'cash on delivery',
        'wallet issue',
        'promo'
    ]):
        return "Customer's Payment Method"
    
    # ========== ADMIN ISSUES ==========
    if any(pattern in text for pattern in [
        'admin issue',
        'admin problem',
        'system error',
        'technical error',
        'platform issue',
        'app issue',
        'offer issue',
        'voucher issue',
        'promo issue'
    ]):
        return 'Admin Issues'
    
    # ========== WRONG BRANCH ==========
    if any(pattern in text for pattern in [
        'wrong branch',
        'different branch',
        'incorrect branch',
        'branch mistake',
        'branch error'
    ]):
        return 'Wrong Branch'
    
    # ========== OTHER ==========
    # Only if nothing else matches
    # Check for truly empty or meaningless
    if len(text) < 3 or text in ['na', 'n/a', '-', 'nil', 'none', 'other', '.', '..']:
        return 'Other'
    
    # If we have substantial text but no pattern matches, still return Other
    # but these should be reviewed
    return 'Other'



In [27]:

def process_file(input_file, output_file):
    """Process the Excel file with comprehensive classification"""
    
    print("Loading data...")
    df = pd.read_excel(input_file)
    print(f"Loaded {len(df)} records")
    
    # Apply classification
    print("Applying comprehensive classification...")
    df['corrected_category'] = df.apply(
        lambda row: classify(row['remarks'], row['cancelledreason']),
        axis=1
    )
    
    # Validate all categories are valid
    invalid_cats = set(df['corrected_category']) - set(valid_categories)
    if invalid_cats:
        print(f"WARNING: Invalid categories found: {invalid_cats}")
    
    # Mark changes
    df['changed'] = df['category'] != df['corrected_category']
    
    # Statistics
    total_changed = df['changed'].sum()
    print(f"\nTotal changes: {total_changed} ({total_changed/len(df)*100:.1f}%)")
    
    # Show all categories being used
    print("\nCategory distribution after correction:")
    category_counts = df['corrected_category'].value_counts()
    for cat, count in category_counts.items():
        original_count = (df['category'] == cat).sum()
        change = count - original_count
        print(f"  {cat}: {count} ({count/len(df)*100:.1f}%) [change: {change:+d}]")
    
    # Check how many different categories are being used
    categories_used = df['corrected_category'].nunique()
    print(f"\nCategories actively used: {categories_used} out of {len(valid_categories)}")
    
    # Show which categories gained the most
    print("\nTop category improvements:")
    improvements = {}
    for cat in valid_categories:
        original = (df['category'] == cat).sum()
        corrected = (df['corrected_category'] == cat).sum()
        if corrected > original:
            improvements[cat] = corrected - original
    
    for cat, improvement in sorted(improvements.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {cat}: +{improvement}")
    
    # Save to Excel
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # All data
        df.to_excel(writer, sheet_name='All_Corrected', index=False)
        
        # Changes only
        changes_df = df[df['changed']]
        changes_df.to_excel(writer, sheet_name='Changes_Only', index=False)
        
        # Category summary
        summary_data = []
        for cat in valid_categories:
            original = (df['category'] == cat).sum()
            corrected = (df['corrected_category'] == cat).sum()
            summary_data.append({
                'Category': cat,
                'Original_Count': original,
                'Corrected_Count': corrected,
                'Change': corrected - original,
                'Percentage': f"{corrected/len(df)*100:.1f}%"
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Category_Summary', index=False)
        
        # Examples of changes for each category
        examples_data = []
        for cat in valid_categories:
            # Find examples where this category was assigned
            examples = df[(df['corrected_category'] == cat) & df['changed']].head(2)
            for _, row in examples.iterrows():
                examples_data.append({
                    'New_Category': cat,
                    'Original_Category': row['category'],
                    'Remarks': str(row['remarks'])[:100],
                    'Cancel_Reason': row['cancelledreason']
                })
        
        if examples_data:
            examples_df = pd.DataFrame(examples_data)
            examples_df.to_excel(writer, sheet_name='Change_Examples', index=False)
    
    print(f"\n✅ Results saved to {output_file}")
    print("\nSheets created:")
    print("  1. All_Corrected - All records with corrections")
    print("  2. Changes_Only - Records that changed")
    print("  3. Category_Summary - Distribution of all categories")
    print("  4. Change_Examples - Sample changes for each category")
    
    return df

In [ ]:
r = process_file(input_file='input.xlsx', 
            output_file='output.xlsx')